In [ ]:
pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [ ]:
print(os.getcwd())

In [ ]:
Oststki = pd.read_excel('Остатки.xlsx', header=10)

# Заменили пустые значения нулями
Oststki = Oststki.fillna(0)

In [ ]:
# Убираем ненужные столбцы
Oststki.drop(columns = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace = True)

# переименуем колонки
Oststki.rename(columns = {'Номенклатура, № по кат., Производитель':'Name', 
                          'Количество (в ед. хранения)':'Нач_ост_кол',
                          'Сумма (в рег. валюте)':'Нач_ост_сумм',
                          'Количество (в ед. хранения).1':'Приход_Кол',
                          'Сумма (в рег. валюте).1':'Приход_Сумм',
                          'Количество (в ед. хранения).2':'Расход_кол',
                          'Сумма (в рег. валюте).2':'Расход_сумм',
                          'Количество (в ед. хранения).3':'Кон_ост_кол',
                          'Сумма (в рег. валюте).3':'Кон_ост_сумм',
                          }, inplace = True )

Oststki.head(10)

# Угданский ЧАТЦ

In [ ]:
# Отберем данные для отдельной таблице по Угданскому Чатц
start = Oststki.index [Oststki['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end = Oststki.index [Oststki['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

In [ ]:
# Функция для отбора только данным по Угданский Чатц
list = [i for i in range(start, end+1)]
Ugdan = Oststki.take(list, axis = 0)

In [ ]:
Ugdan.tail()

## Данные (масла)

In [ ]:
# Отберем данные для отдельной таблицы по Угданскому Чатц по маслам
start_oil = Ugdan.index [Ugdan['Name']== "Угданский ЧАТЦ  склад " ]. tolist ()[0]
end_oil = Ugdan.index [Ugdan['Name']== "Материалы, , " ]. tolist ()[0]

list_oil = [i for i in range(start_oil - start, end_oil-start)]

Ugdan_oil = Ugdan.take(list_oil, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
Ugdan_oil = Ugdan_oil[~Ugdan_oil.Name.str.contains('|'.join(rule))]

# Уберем ненужные строки
Ugdan_oil = Ugdan_oil.query("Name not in ['Угданский ЧАТЦ  склад ']")

Ugdan_oil.tail()

In [ ]:
# Подсчитаем нужные метрики

Quanity_oil = Ugdan_oil['Кон_ост_кол'].sum()
Sum_oil = Ugdan_oil['Кон_ост_сумм'].sum()
Sales_oil = Ugdan_oil['Расход_кол'].sum()
Sales_sum_oil = Ugdan_oil['Расход_сумм'].sum()
Sum_begin_oil = Ugdan_oil['Нач_ост_сумм'].sum()

## Данные (товары и запчасти)

In [ ]:
start_prod = Ugdan.index [Ugdan['Name']== "Материалы, , " ]. tolist ()[0]
end_prod = Ugdan.index [Ugdan['Name']== "Основной ЧАТЦ" ]. tolist ()[0]

list_prod = [i for i in range(start_prod-start, end_prod-start)]

Ugdan_prod = Ugdan.take(list_prod, axis = 0)

# Исключим из таблицы строки, где Name включает в себя ', ,'
rule = [", ,"]
Ugdan_prod = Ugdan_prod[~Ugdan_prod.Name.str.contains('|'.join(rule))]

Ugdan_prod.tail()

In [ ]:
# Подсчитаем нужные метрики

Quanity_prod = Ugdan_prod['Кон_ост_кол'].sum()
Sum_prod = Ugdan_prod['Кон_ост_сумм'].sum()
Sales_prod = Ugdan_prod['Расход_кол'].sum()
Sales_sum_prod = Ugdan_prod['Расход_сумм'].sum()
Sum_begin_prod = Ugdan_prod['Нач_ост_сумм'].sum()

# Итоговые значения по метрикам

In [ ]:
metrics = pd.DataFrame({'Склад': ['Угданский - масло', 'Угданский - запасные части', 'Итого', 'Продажи за ГОД в ценах с/с', 'Коэфф. оборачиваемости товара'], 
                    'Шт.': [Quanity_oil, Quanity_prod, Quanity_oil+Quanity_prod, Sales_oil+Sales_prod, '0'], 
                    'Сумма.': [Sum_oil, Sum_prod, Sum_oil+Sum_prod, Sales_sum_oil+Sales_sum_prod, (Sales_sum_oil+Sales_sum_prod)/((Sum_begin_oil+Sum_begin_prod+Sum_prod+Sum_oil)/2)]})
metrics = metrics.set_index('Склад').round(2)

metrics

In [ ]:
metrics.to_excel ('Metrics.xlsx')